In [4]:
#gzoltar_files_path = "/Users/lorenapacheco/Concordia/Masters/BugReportsMining/gzoltar_files"
gzoltar_files_path = r"C:\Users\loren\Concordia\Masters\BugReportsMining\gzoltar_files"
#out = '/Users/lorenapacheco/Concordia/Masters/BugReportsMining/ochiaiScores2.0/Ochiai4/'
out = r"C:\Users\loren\Concordia\Masters\BugReportsMining\ochiaiScores2.0\Ochiai4/"
#data_file_path = "/Users/lorenapacheco/Concordia/Masters/BugReportsMining/data/merged_data_production_bug_reports.json"
data_file_path = r"C:\Users\loren\Concordia\Masters\BugReportsMining\data\merged_data_production_bug_reports.json"
fake_failing_tests_json = 'fake_failing_tests_info'

## Reading the coverage data
Using the files per method and without the tests from the buggyfix commit
These tests were generated in the Ochiai1 script

In [5]:
import csv
import re
import os
import json


def json_file_to_dict(file):
    data = {}
    with open(file, 'r') as fp:
        data = json.load(fp)
    return data

def read_methods_matrix_file(file_path):
    statements_covered_per_test = []
    test_passed = []

    with open(os.path.join(file_path, "methods_matrix.txt"), 'r') as f:
        for line in f:
            row = [int(num) for num in line.strip().split()]
            statements_covered_per_test.append(row)
    return statements_covered_per_test

def read_methods_spectra_file(file_path):
    lines_of_code_obj_list = []
    with open(os.path.join(file_path, "methods_spectra.csv"), 'r') as file:
        first_line = True
        for line in file:
            # Skip the first line
            if first_line:
                first_line = False
                continue
            lines_of_code_obj_list.append(line.replace("\n", ""))
    return lines_of_code_obj_list

def read_tests_csv_to_lists(file_path):
    with open(os.path.join(file_path, "fake_test_results_ochiai_1.csv"), 'r') as csvfile:
        reader = csv.reader(csvfile)
        test_names = []
        # test_results = []
        for row in reader:
            test_names.append(row[0].replace("\n", ""))  # First column
            # test_results.append(row[1])  # Only the test names interest me
    #return test_names, test_results
    return test_names


bugs_data = json_file_to_dict(data_file_path)
coverage_data = {}

for project in bugs_data.keys():
    project_gzoltar_folder = os.path.join(gzoltar_files_path, project)
    if not os.path.exists(project_gzoltar_folder):
        print("Gzoltar folder not found for the project " + project)
        print("Skipping!!!!! ")
        continue
    if project not in coverage_data.keys():
        coverage_data[project] = {}
    for bug_id in bugs_data[project].keys():
        bug_gzoltar_folder = os.path.join(project_gzoltar_folder, bug_id)
        if not os.path.exists(bug_gzoltar_folder):
            print("Gzoltar folder not found for the bugId " + project + "-" + bug_id)
            print("Skipping!!!!! ")
            continue
        coverage = {}
        try:
            coverage["methods_covered_per_test"] = read_methods_matrix_file(bug_gzoltar_folder)
            coverage["methods_obj_list"] = read_methods_spectra_file(bug_gzoltar_folder)
            test_names = read_tests_csv_to_lists(bug_gzoltar_folder)
            coverage["test_names"] = test_names
            print("Number of tests in bug " + project + "-" + bug_id + " - " + str(len(test_names)))
            coverage["test_results"] = test_results = [True] * len(test_names) # Setting all the tests as passing until we define the fake failing tests
            coverage_data[project][bug_id] = coverage
        except FileNotFoundError:
            print ("The bug " + project + "-" + bug_id + " does not contain one the the required files. Skipping it")
            continue

print("Done")

Number of tests in bug Cli-14 - 543
Number of tests in bug Cli-5 - 99
The bug Closure-106 does not contain one the the required files. Skipping it
Number of tests in bug Closure-125 - 8157
The bug Closure-143 does not contain one the the required files. Skipping it
Number of tests in bug Closure-152 - 0
Number of tests in bug Closure-2 - 7827
The bug Closure-27 does not contain one the the required files. Skipping it
Number of tests in bug Closure-34 - 7413
Number of tests in bug Closure-37 - 7372
Number of tests in bug Closure-55 - 7167
Number of tests in bug Codec-8 - 303
Number of tests in bug Collections-28 - 5283
Number of tests in bug Compress-1 - 69
Number of tests in bug Compress-12 - 315
Number of tests in bug Compress-14 - 319
Number of tests in bug Compress-17 - 350
Number of tests in bug Compress-18 - 356
Number of tests in bug Compress-23 - 465
Number of tests in bug Compress-32 - 612
Number of tests in bug Csv-12 - 197
Number of tests in bug Csv-4 - 177
Number of tests in

In [29]:
import glob
import os
import json
import math


def dict_to_json_file(file, dic, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    with open(os.path.join(folder, file+'.json'), 'w') as fp:
        json.dump(dic, fp, sort_keys=True, indent=4)
    fp.close()

def write_two_lists_to_csv(list1, list2, filename):
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        for item1, item2 in zip(list1, list2):
            writer.writerow([item1, item2])

def store_fake_test_results(coverage, project, bug_id):
    project_gzoltar_folder = os.path.join(gzoltar_files_path, project)
    bug_gzoltar_folder =  os.path.join(project_gzoltar_folder , bug_id)
    test_names = coverage["test_names"]
    fake_test_results_ochiai_2 = coverage["fake_test_results"]
    fake_test_results_ochiai_2_file_name =  os.path.join(bug_gzoltar_folder, "fake_test_results_ochiai_2.csv")
    write_two_lists_to_csv(test_names, fake_test_results_ochiai_2, fake_test_results_ochiai_2_file_name)

def find_file_complete_name(file, bug_data):
    for file_complete_name in bug_data["stackTraceMethodsDetails"].keys():
        if file_complete_name.endswith(file):
            return file_complete_name
    return None

def get_top_n_keys(dictionary, keys_to_consider, n):
    sorted_dict = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    filtered_dict = [(key, value) for key, value in sorted_dict if key in keys_to_consider]
    top_n_pairs = filtered_dict[:n]
    top_n_keys = [pair[0] for pair in top_n_pairs]
    return top_n_keys

bugs_data = json_file_to_dict(data_file_path)
fake_failing_tests_info = {}
for project in coverage_data.keys():
    for bug_id in coverage_data[project].keys():
        bug_data = bugs_data[project][bug_id]
        coverage = coverage_data[project][bug_id]

        print(project + " ---- " + bug_id)
        buggy_commit = bug_data["buggy_commit"]
        methods_list = []

        if bug_data["stackTraceMethodsDetails"] =={ }:
            print("The bug does not contain stackTraceMethodsDetails. Skipping.")
            continue

        print("* Part 1 - defining the fake failing tests")

        stack_trace_methods_first_5 = bug_data["stack_trace_methods"][:5]
        stack_trace_files_first_5 = bug_data["stack_trace_files"][:5]
        stack_traces_methods_test_count = {}
        num_lines_covered_per_test = {}
        for index,st_file in enumerate(stack_trace_files_first_5):
            st_method = stack_trace_methods_first_5[index].split(".")[-1]
            st_file_complete_name = find_file_complete_name(st_file, bug_data)
            if st_file_complete_name:
                if st_method in bug_data["stackTraceMethodsDetails"][st_file_complete_name].keys() and "tests_covering_the_method" in bug_data["stackTraceMethodsDetails"][st_file_complete_name][st_method].keys():
                    for test in bug_data["stackTraceMethodsDetails"][st_file_complete_name][st_method]["tests_covering_the_method"]:
                        num_lines = len(bug_data["stackTraceMethodsDetails"][st_file_complete_name][st_method]["tests_covering_the_method"][test])
                        if test in stack_traces_methods_test_count.keys():
                            stack_traces_methods_test_count[test] += 1
                            num_lines_covered_per_test[test] += len(bug_data["stackTraceMethodsDetails"][st_file_complete_name][st_method]["tests_covering_the_method"][test])
                        else:
                            stack_traces_methods_test_count[test] = 1
                            num_lines_covered_per_test[test] = len(bug_data["stackTraceMethodsDetails"][st_file_complete_name][st_method]["tests_covering_the_method"][test])

        if stack_traces_methods_test_count== {}:
            print("No tests found covering the stack traces top 5 methods. Skipping this bug")
            continue

        threshold_lines = max(num_lines_covered_per_test.values())
        print("threshold_lines: " + str(threshold_lines))
        tests_that_cover_more_lines = []
        stack_traces_methods_test_count_above_threshold_lines = {}
        for test in num_lines_covered_per_test.keys():
            if num_lines_covered_per_test[test] >= threshold_lines:
                tests_that_cover_more_lines.append(test)
                stack_traces_methods_test_count_above_threshold_lines[test] = stack_traces_methods_test_count[test]

        threshold_st_methods= max(stack_traces_methods_test_count_above_threshold_lines.values())
        print("threshold_tests: " + str(threshold_st_methods))
        selected_tests = []
        for test in stack_traces_methods_test_count_above_threshold_lines.keys():
            if stack_traces_methods_test_count_above_threshold_lines[test] >= threshold_st_methods:
                selected_tests.append(test)

        print("* Part 2 - preparing the fake_tests_status")
        coverage["fake_test_results"] = coverage["test_results"]
        fake_failed_tests_number = 0
        for index, test in enumerate(coverage["test_names"]):
            if test in selected_tests:
                fake_failed_tests_number +=1
                coverage["fake_test_results"][index] = False

        print("Storing the fake test results from this implementation in a file for future uses")
        store_fake_test_results(coverage, project, bug_id)
        print("store_fake_test_results operation completed")


        print("* Part 3 - Executing Ochiai")
        methods_ochiai_scores = {}
        for index_m, method_name in enumerate(coverage["methods_obj_list"]):
            n00 = 0
            n01 = 0
            n10 = 0
            n11 = 0
            s_o = 0
            for index_t, test_name in enumerate(coverage["test_names"]):
                if str(coverage["methods_covered_per_test"][index_t][index_m]) == "1":
                    if not coverage["fake_test_results"][index_t]:
                        n11 += 1
                    else:
                        n10 += 1
                else:
                    if not coverage["fake_test_results"][index_t]:
                        n01 += 1
                    else:
                        n00 += 1
            try:
                s_o = n11/math.sqrt((n11+n01)*(n11+n10))
            except ZeroDivisionError:
                s_o = 0
            methods_ochiai_scores[method_name] = s_o

        if project not in fake_failing_tests_info.keys():
            fake_failing_tests_info[project] = {}

        fake_failing_tests_info[project][bug_id] = {}
        fake_failing_tests_info[project][bug_id]["fake_passing_tests_number"] = len(coverage["fake_test_results"]) - fake_failed_tests_number
        fake_failing_tests_info[project][bug_id]["fake_failing_tests_number"] = fake_failed_tests_number

        print("Number of fake passing tests: " + str(fake_failing_tests_info[project][bug_id]["fake_passing_tests_number"]))
        print("Number of fake failing tests: " + str(fake_failed_tests_number) + "\n")
        dict_to_json_file(bug_id, methods_ochiai_scores, out + project + "/") # Everything is 0. Try to understand

dict_to_json_file(fake_failing_tests_json, fake_failing_tests_info, out)
print("Execution completed")

Cli ---- 14
* Part 1 - defining the fake failing tests
threshold_lines: 31
threshold_tests: 4
* Part 2 - preparing the fake_tests_status
Storing the fake test results from this implementation in a file for future uses
store_fake_test_results operation completed
* Part 3 - Executing Ochiai
Number of fake passing tests: 535
Number of fake failing tests: 8

Cli ---- 5
* Part 1 - defining the fake failing tests
threshold_lines: 14
threshold_tests: 3
* Part 2 - preparing the fake_tests_status
Storing the fake test results from this implementation in a file for future uses
store_fake_test_results operation completed
* Part 3 - Executing Ochiai
Number of fake passing tests: 95
Number of fake failing tests: 4

Closure ---- 125
* Part 1 - defining the fake failing tests
threshold_lines: 78
threshold_tests: 5
* Part 2 - preparing the fake_tests_status
Storing the fake test results from this implementation in a file for future uses
store_fake_test_results operation completed
* Part 3 - Executing 